# Data Arts Module 2: Social Network Evolution

## Intro

In this module, we will write code that allows us to visualize the evolution of a social network over time. We load data out of CSV, which represents people meeting each other and then losing contact with them, then we will plot that data on a graph that shows us the network as it evolves. 

In order to create this visualization, we are going to need to understand a few principles of the Python programming language. Before you proceed, read the three notes inside the "Concepts in Programming" folder. These will familiarize you with the topics of object oriented programming, list and dictionary comprehensions, and functional programming.

## 1. Importing Libraries

Lucky for us, other data scientists also like plotting data in Python. They have already written large compilations of code, called libraries, which handle making and drawing graphs in Python. Since these libraries are published online, we have access to all that hard work too! That means we can use code from those libraries to handle drawing the graph, so we only need to worry about loading our data into it.

You don't need to worry too much about the code in the next cell. It's purpose is to import libraries that other people have written, so that we have access to them later on.

In [10]:
import matplotlib.pyplot as plt # Helps us draw graphs.
import ipywidgets as widgets    # Helps us make graphs that change, like with a slider.
%matplotlib inline

import csv                      # Helps us read data from a CSV file.
import copy                     # Helps us copy Python objects without weird errors.
import networkx as nx           # Helps us make the actual network.
from random import random       # Helps us generate random numbers.

size = plt.rcParams["figure.figsize"]

In [11]:
class Network(nx.DiGraph):
    """Network class."""
    
    # Initialize a new Network.
    def __init__(self):
        """Initialize a new Network as we would initialize a DiGraph, or directed graph. The
        new Network needs a population, which starts out empty because we haven't added any
        people yet!
        """
        nx.DiGraph.__init__(self)
        self.population = {}

    def add_node(self, person):
        """This function will add a new Person into the Network! We'll use the `nx.DiGraph`
        library we imported earlier, so it can handle all the complicated stuff. We just need
        to tell it that the value of the new node is the person that this function takes as
        input. This also means the population of the Network has been modified, so let's add
        the person into the population dictionary too, mapping from the person's id to the
        actual Person object.
        """
        nx.DiGraph.add_node(self, person)
        self.population[person.id] = person

class Person:
    """Person class."""
    
    empty = set() # When someone has no connections, we'll refer to this empty set.

    def __init__(self, id, conn1, conn2, conn3, start, duration, end):
        """Initialize a new Person. We'll take as input all the data that we gathered from the
        CSV. This includes an id, 3 connections, a start time, a duration of the friendship,
        and an end time. All this info should be stored in the new Person object that we're
        making. Notice that we store the Person's connections not as 3 variables, but rather
        as a set containing 3 elements. This is just in case we want to give a Person more than
        3 connections in the future. We also give each Person a `pos`, or position. This is a
        random (x, y) coordinate within the size of the graph.
        """
        self.id = id
        self.pos = (random() * size[0], random() * size[1])
        self.potential_connections = set((conn1, conn2, conn3))
        self.start = start
        self.duration = duration
        self.end = end

    def connections(self, step):
        """Get a set containing all the connections the Person has at a particular time step.
        If the time is between the start and end of the friendship, then we'll return the
        connections the Person has. Meanwhile, if the friendship hasn't started, or if it has
        already ended, then we will return the empty set of no connections.
        """
        if self.start <= step < self.end:
            return self.potential_connections
        return Person.empty

In [12]:
def init_network(file_name):
    """This function will initialize a Network full of People representing the data in a CSV.
    First create a totally blank Network. Then read each row in the designated CSV file. For
    each row, create a new Person using the info in that row. Then add that Person into the
    blank Network. When it's all done and every Person has been added, return the Network,
    now containing all the people described in the CSV.
    """
    network = Network() # Initialize a blank Network.
    with open(file_name) as file: 
        # Open the CSV file, and make a `reader` that can understand the data inside.
        reader = csv.reader(file, delimiter=',')
        labels = next(reader)
        # Then, for every row in the file ...
        for row in reader:
            row = [int(param) for param in row] # Make a list containing the data from the row.
            person = Person(*row) # Then use that data to make a new Person.
            network.add_node(person) # Finally, add the new Person into our blank Network.
    return network

blank_network = init_network('network.csv') # Make a Network using our specific CSV.

In [20]:
def get_frame(step):
    """This function will draw a Network that contains only the connections that should be
    active at the given time step. ie, it only shows the friendships that are currently alive.
    """
    # First, add all the edges that should be included in the Network at this time step.
    network = copy.deepcopy(blank_network)
    for person in network.nodes(): # For every person ...
        for other in person.connections(step): # And every one of their current friends ...
            network.add_edge(person, network.population[other]) # Add an edge connecting them.
    # Then, draw the Network.
    # `pos` maps each Person to an (x, y) coordinate.
    # `node_connections` is how many connections each node has.
    # We use `node_connections` to give each node a color (bluer = fewer, redder = more).
    pos = {person: person.pos for person in network.nodes()}
    node_connections = [len(person.connections(step)) for person in network.nodes()]
    # This stuff below draws a pretty graph, using the libraries we imported before.
    # It's not important to know how these work, since you need knowledge of those libraries.
    plt.figure(3, figsize=(16, 16)) 
    nx.draw(network, pos, node_size=24, cmap=plt.get_cmap('jet'), node_color=node_connections,
            width=0.8)
    plt.show()

In [23]:
# This line adds the slider to the graph using Python widgets.
_ = widgets.interact(get_frame, step=widgets.IntSlider(min=0, max=300, value=0))

# NOTE:
# You may see a red error box, upon executing this line.
# That's chill, it's just a warning for the people who made one of the libraries we're using.

Play around with the network by stepping it forward and backwards through time, and see how the different connections evolve! You can right-click on the visualization at any time step to save the image, if you like.

As you can see, we've created an awesome prototype of basic social network evolution. Review the lab and see how the programming methods explained at the top of the module were used in creating the visualization. See if you can play around and create your own network visualizations with different parameters!